# ejercicio CIFAR10 con tensorflow 1.4 o superior

In [108]:
import tensorflow as tf
import numpy as np
import pandas as pd

[(x_train, y_train), (x_test, y_test)]=tf.keras.datasets.cifar10.load_data ()


In [109]:
num_classes=10
y_train = tf.keras.utils.to_categorical(y_train, num_classes).astype(np.float32)
y_test = tf.keras.utils.to_categorical(y_test, num_classes).astype(np.float32)

In [110]:
print('x train:',x_train.shape)
x_train=x_train/255
x_test=x_test/255
#pd.get_dummies(y_train)
print('y train:',y_train.shape)
num_labels=10
print('x test:',x_test.shape)
print('y test:',y_test.shape)


x train: (50000, 32, 32, 3)
y train: (50000, 10)
x test: (10000, 32, 32, 3)
y test: (10000, 10)


In [111]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [114]:
batch_size = 16
image_size= 32
patch_size=5
depth = 32
depth2=64

num_hidden = 4096
num_hidden_2=1024
num_channels=3


graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
 # tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(x_test,dtype=tf.float32)
  #tf_test_dataset = tf.placeholder(
   # tf.float32, shape=(batch_size, image_size, image_size, num_channels))  
  # Variables.
#5*5*3*32
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
#5*5*32*64
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth2], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))

# fully connected layers
#4096, 1024
  layer3_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_hidden_2], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_2]))
#1024,10
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden_2, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

#keep prob
  keep_prob=tf.placeholder(tf.float32)


 
  # Model.
  def model(data,keep_prob):
   #def model(data,keep_prob):
        #conv 1
        #32*32*3*32
    conv = tf.nn.conv2d(data, layer1_weights, strides=[1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    
    #pooling
    #16*16*32*32
    pool_1=tf.nn.max_pool(hidden,[1,2,2,1],[1,2,2,1],padding='SAME')
    
    # Dropout
    #pool_1_drop=tf.nn.dropout(pool_1,keep_prob)
    
    #conv 2
    #16*16*32*64
    conv = tf.nn.conv2d(pool_1, layer2_weights, strides=[1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    
    #pooling
    #8*8*64*64
    pool_2=tf.nn.max_pool(hidden,[1,2,2,1],[1,2,2,1],padding='SAME')
    # Dropout
    #pool_2_drop=tf.nn.dropout(pool_2,keep_prob)
    
    #conv 3
    #conv = tf.nn.conv2d(pool_2, layer3_weights, strides=[1, 1, 1, 1], padding='SAME')
    #hidden = tf.nn.relu(conv + layer3_biases)
    
    #pooling

    
    # Fully connected
    
    shape = pool_2.get_shape().as_list()
    reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    #hidden = tf.nn.relu(tf.matmul(hidden, layer5_weights) + layer5_biases)
        
    hidden=tf.nn.dropout(hidden,keep_prob)
    return tf.matmul(hidden, layer4_weights) + layer4_biases


## same model without dropout for testing
  # Model.
 

   # Training computation.
  #logits = model(tf_train_dataset)
  logits = model(tf_train_dataset,keep_prob=0.5) 
  loss = tf.reduce_mean(
  tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  # regulizer
  #regularizers=tf.nn.l2_loss(layer1_weights)+tf.nn.l2_loss(layer2_weights)+tf.nn.l2_loss(layer3_weights)+tf.nn.l2_loss(layer4_weights)+tf.nn.l2_loss(layer5_weights)
  #loss=tf.reduce_mean(loss+beta*regularizers)
  #loss=tf.reduce_mean(loss)
  # Optimizer.
  # agregando un learning rate descendiente
  #global_step=tf.Variable(0)
  #start_learning_rate=0.5
  #learning_rate=tf.train.exponential_decay(start_learning_rate,global_step,100000,0.96,staircase=True)
  learning_rate=1e-3  
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
  optimizer=tf.train.AdamOptimizer(learning_rate).minimize(loss) 
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  #valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  #test_prediction = tf.nn.softmax(model(tf_test_dataset))    
    
 
  test_prediction = tf.nn.softmax(model(tf_test_dataset,keep_prob=1.0))


In [115]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (y_train.shape[0] - batch_size)
    batch_data = x_train[offset:(offset + batch_size), :, :, :]
    batch_labels = y_train[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keep_prob:0.5}
    #feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      #print('Validation accuracy: %.1f%%' % accuracy(
       # valid_prediction.eval(), valid_labels))
        
  
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), y_test))

Initialized
Minibatch loss at step 0: 33.536911
Minibatch accuracy: 6.2%
Minibatch loss at step 50: 5.111912
Minibatch accuracy: 18.8%
Minibatch loss at step 100: 3.651460
Minibatch accuracy: 12.5%
Minibatch loss at step 150: 3.992448
Minibatch accuracy: 12.5%
Minibatch loss at step 200: 4.229755
Minibatch accuracy: 12.5%
Minibatch loss at step 250: 2.187902
Minibatch accuracy: 18.8%
Minibatch loss at step 300: 3.187607
Minibatch accuracy: 25.0%
Minibatch loss at step 350: 1.441424
Minibatch accuracy: 50.0%
Minibatch loss at step 400: 1.911334
Minibatch accuracy: 31.2%
Minibatch loss at step 450: 2.201990
Minibatch accuracy: 25.0%
Minibatch loss at step 500: 2.018458
Minibatch accuracy: 43.8%
Minibatch loss at step 550: 1.978348
Minibatch accuracy: 37.5%
Minibatch loss at step 600: 1.941598
Minibatch accuracy: 37.5%
Minibatch loss at step 650: 2.154346
Minibatch accuracy: 12.5%
Minibatch loss at step 700: 1.757343
Minibatch accuracy: 25.0%
Minibatch loss at step 750: 1.837918
Minibatch

# problema con otro estilo de programacion.

In [31]:
def weight_variable(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

#funcion helper convulucion con stride 1 y padding same
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#Max pooling stride 2, kernel 2 y padding same
def max_pool_2x2(x):
    return tf.nn.max_pool(x,[1,2,2,1],[1,2,2,1],padding='SAME')

# conv layer

def conv_layer(input,shape):
    W=weight_variable(shape)
    b=bias_variable([shape[3]])
    return tf.nn.relu(conv2d(input,W)+b)
#full layer 
def full_layer(input,size):
    in_size=int(input.get_shape()[1])
    W=weight_variable([in_size,size])
    b=bias_variable([size])
    return tf.matmul(input,W)+b

In [32]:
# entradas
x=tf.placeholder(tf.float32,shape=[None,32,32,3])
y=tf.placeholder(tf.float32,shape=[None,10])
keep_prob=tf.placeholder(tf.float32)


In [33]:
#model 

num_channels=3
patch_size=5
depth=32
depth2=64
num_hidden=1024
batch_size=16

conv1=conv_layer(x,shape=[patch_size,patch_size,num_channels,depth])
conv1_pool=max_pool_2x2(conv1)

conv2=conv_layer(conv1_pool,shape=[patch_size,patch_size,depth,depth2])
conv2_pool=max_pool_2x2(conv2)
conv2_flat=tf.reshape(conv2_pool,[-1,8*8*depth2])

#fully connected
full_1=tf.nn.relu(full_layer(conv2_flat,num_hidden))
full_1_drop=tf.nn.dropout(full_1,keep_prob=keep_prob)

y_conv=full_layer(full_1_drop,10)

# salida
cross_entropy= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_conv))

train_step=tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)
correct_prediction=tf.equal(tf.argmax(y_conv,1),tf.argmax(y,1))
accuracy_model=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
train_prediction=tf.nn.softmax(y_conv)


In [34]:
#def test(sess):
 #   x_test=x_test
  #  y_test=y_test
   # acc=np.mean([sess.run(accuracy,feed_dict={x:x_test[i],y:y_test[i],keep_prob=1}) for i in range(10)])
    #print("Accuracy:{:.4}".format(acc*100)
          

In [35]:
num_steps = 1001

with tf.Session() as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (y_train.shape[0] - batch_size)
    batch_data = x_train[offset:(offset + batch_size), :, :, :]
    batch_labels = y_train[offset:(offset + batch_size), :]
    feed_dict = {x : batch_data, y : batch_labels,keep_prob:0.5}
    _, l, predictions = session.run(
      [train_step, cross_entropy, train_prediction], feed_dict=feed_dict)
    #session.run(train_step,feed_dict = {x : batch_data, y : batch_labels,keep_prob:0.5})
    
      #[optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))

Initialized
Minibatch loss at step 0: 16.729513
Minibatch accuracy: 18.8%
Minibatch loss at step 50: 2.324462
Minibatch accuracy: 18.8%
Minibatch loss at step 100: 1.842644
Minibatch accuracy: 37.5%
Minibatch loss at step 150: 2.045403
Minibatch accuracy: 25.0%
Minibatch loss at step 200: 2.259174
Minibatch accuracy: 6.2%
Minibatch loss at step 250: 1.991985
Minibatch accuracy: 25.0%
Minibatch loss at step 300: 1.627606
Minibatch accuracy: 50.0%
Minibatch loss at step 350: 1.518497
Minibatch accuracy: 37.5%
Minibatch loss at step 400: 1.879521
Minibatch accuracy: 25.0%
Minibatch loss at step 450: 1.573216
Minibatch accuracy: 31.2%
Minibatch loss at step 500: 1.971918
Minibatch accuracy: 31.2%
Minibatch loss at step 550: 1.567602
Minibatch accuracy: 37.5%
Minibatch loss at step 600: 1.941336
Minibatch accuracy: 31.2%
Minibatch loss at step 650: 2.083199
Minibatch accuracy: 37.5%
Minibatch loss at step 700: 1.505296
Minibatch accuracy: 50.0%
Minibatch loss at step 750: 1.886604
Minibatch